In [41]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import plotly.express as px
import re


## 數據前處理階段

In [2]:
sales = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='出貨明細') # Mac使用
# sales = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='出貨明細')
# budget = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F') # Mac使用
# budget = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F')

In [69]:
def categorize_product(data):
    return re.search('[A-Z]*', data).group(0)
#     print(re.search('[0-9]+', data).group())

# categorize_product('ACA-2012-A1-CC-S成')

In [63]:
test = sales[['品名', '產品分類']]

In [70]:
test['Type'] = test['產品分類'].apply(categorize_product)

<ipython-input-70-bef036d71eef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Type'] = test['產品分類'].apply(categorize_product)


In [115]:
re.search('/d*', 'AAF成品')

In [130]:
re.search('[0-9]+', 'WIP2520 P-1 Series').group(0)

'2520'

In [139]:
test['產品分類'].str.extract(r'[A-Z]*')

ValueError: pattern contains no capture groups

In [38]:
sales['產品分類'].unique()

array(['WIP2520 P-1 Series', 'MLVS0603 AM成品', 'MLVS0603 L Free Ser',
       'WIP2016 P-1 Series', 'MIP2520 series成品', 'MLVS0201成品',
       'EGA0603成品', 'EGA0402AH成品', 'MLVS1206 AM成品', 'SGX0603成品',
       'WIP2520 S-1 Series', 'MLVS0805 AM成品', 'TVS 0402',
       'MCI1005 series成品', 'MCI0603 series成品', 'MGB1005-G series成',
       'MLVG0402 series成品', 'SGX1206-4成品', 'MLVS0402 AM成品',
       'MLVG0603 series成品', 'MLA0402series成品', 'MCB1005-B series成',
       'WIP2520 P Series成', 'MCI1608 series成品', 'WIP2016 S-1 Series',
       'MHC4516-S series成', 'MHC1608-P series成', 'MCM2012-B series成',
       'EGA0402成品', 'MLVS0402 L Free Ser', 'WIP Series 成品', 'EGA1206-4成品',
       'MLV0402成品', 'MLA0603series成品', 'TVS 0201 T成品', 'MLV0805-4成品',
       'MIP2016-P series成', 'MHC1608-S series成', 'MHC2012-S series成',
       'MHC1005-S series成', 'MLVS0805 AM W成品', 'TVS 0201', 'MISC',
       'MLV1206 series成品', 'SMXX  A Series 成品', 'MHC3216-S series成',
       'MHC1005-P series成', 'MCB1608-S series成', 'MCB1005-

In [ ]:
budget.insert(2, 'BU', value=budget.apply(lambda x: 'RF' if 'RF' in x['客戶名稱'] else 'Component', axis=1))

In [ ]:
budget['客戶名稱'] = budget['客戶名稱'].str.replace(' RF', '')

In [ ]:
client_dict = {'Bourns': 'Bourns', 'Eaton': 'Eaton', 'Laird': 'Laird', 'LF': 'LF', 'LF-OEM': 'LF ORM', 'WE': 'WE',
               '信昌': '信昌', '東莞弘電': '東莞弘電', '藝感': '藝感', '華科': '華新科技',
               'Mt System(Component)': 'MT-SYSTEM', 'Teltonika(Component)': 'TELTONIKA', 'Mt System(RF)': 'MT-SYSTEM',
               'Teltonika(RF)': 'TELTONIKA', 'Te(RF)': 'TE', 'Credence(India)': 'Credence',
               'Millenium(India)': 'Millenium', 'Nexty(India)': 'Nexty', 'OSE(India)': 'OSE', 'Others/One Time PO(India)': 'Others/One Time PO(India)'}
budget['客戶名稱'] = budget['客戶名稱'].map(client_dict)

In [ ]:
sales = sales[~sales['狀態'].str.contains('待通知 | 合約')]

## Treemap

In [ ]:
test = sales.groupby(['預交年份', '負責業務', 'Group', 'BG', '預交月份'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
fig = px.treemap(test, path=['預交年份', '負責業務', 'Group', 'BG', '預交月份'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

In [ ]:
test1 = test.query('Group == ["TELTONIKA", "MT-SYSTEM"]')
fig = px.treemap(test1, path=['預交年份', 'Group', 'BG'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

## Line plot

In [ ]:
line_plot = test.groupby(['預交年份', '預交月份', '負責業務'])[['本國幣別NTD']].sum().reset_index()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
line_plot['預交月份'] = pd.Categorical(line_plot['預交月份'], ordered=True, categories=month_order)
line_plot = line_plot.sort_values(by='預交月份')

In [ ]:
line = px.line(line_plot, x='預交月份', y='本國幣別NTD', color='預交年份', category_orders={'預交月份':month_order}, facet_col='負責業務')
line.show()

## Bar plot

In [ ]:
df = test.query('預交年份 == (2020, 2019, 2021)')
df = df.groupby(['預交年份', '負責業務', '預交月份'])[['本國幣別NTD']].sum().reset_index()
df['預交年份'] = df['預交年份'].astype(str)
bar = px.bar(df, x='預交月份', y='本國幣別NTD', color='預交年份', barmode='group',
             category_orders={'預交月份':month_order}, facet_col='負責業務'
            )
bar.show()